# 極性判定とDoc２Vecを使ったTwitterネガポジ予測（実行結果）

## データセットの取得

In [ ]:
from get_dataset import Get_Twitter

#データセットの取得
GT = Get_Twitter()
similar = GT.main()

## 前処理、学習

In [3]:
from preprocessing import For_Model
from model import Model

#前処理
FM = For_Model()
df_tweet, similar = FM.main()

#学習
M = Model()
m, trainings = M.main()

ツイート予定文書を入力してください
朝の電車で並んでた列に割り込むのも、降りてる人待ってるのに押してくるのも、私は60過ぎた男女にしかされたことないんだけど、「最近の若者は電車のマナーが悪い」っていうのはどの世代が言ってるの？？
前処理中です…
読み込んだツイート数 1302
csv出力完了：train_buzz.txt
学習用ツイート数（判定用ツイート含む/短すぎるツイートは削除）： 1045
[分かち書きサンプル]
 絶対 断ら ない 評判 病児保育 助成金 下り 赤字 出し 閉鎖 全国 病児保育 赤字 運営 おり 東海 キッズケア 助成金 求め 署名 集め 助成金 下りる あり ませ 社会保障 税金 使わ ませ 

Doc2Vecで学習しています…
学習が完了しました


## 推定

In [4]:
import predict

#推定
if __name__ == '__main__':
    predict.main(m, trainings, similar, df_tweet)

============== ツイート予定文 ==============

朝の電車で並んでた列に割り込むのも、降りてる人待ってるのに押してくるのも、私は60過ぎた男女にしかされたことないんだけど、「最近の若者は電車のマナーが悪い」っていうのはどの世代が言ってるの？？

======= 類似ツイート上位１０（全1045ツイート中） =======

※類似度0.55未満は表示しない
※nega/posiスコアは対数変換したフォロワー数で割っています

…………　類似ツイート1位：類似度 0.99　…………

朝の電車で並んでた列に割り込むのも、降りてる人待ってるのに押してくるのも、私は60過ぎた男女にしかされたことないんだけど、「最近の若者は電車のマナーが悪い」っていうのはどの世代が言ってるの？？🤔

【極性】： negative
posi_score： 6.1 ／ nega_score： 7.8 ／followers： 116320


…………　類似ツイート2位：類似度 0.64　…………

「クルマ社会と電車社会の境目はどこにあるんだろう？」と思い、簡単に可視化してみた。

2010年国勢調査のデータより、【15歳以上の男性が通勤で使う交通手段1位】で塗り分け。
複数回答なので、例えば「駅まで車・駅から電車」の人は自家用車と電車の両方にカウントされるのでご注意を。 

【極性】： positive
posi_score： 9.9 ／ nega_score： 3.2 ／followers： 6267


…………　類似ツイート3位：類似度 0.61　…………

昔の話なんやけど、私の同僚は毎日電車で化粧する人やってん。
同じ電車に乗る私も他の同僚もみっともないからやめるように何回も注意したんやけど、案の定聞き入れてもらえへんかった。
ある日また同じ様に化粧してた彼女がとある男性の服を汚してしまった。
相手は一張羅を汚されて激怒したけど、

【極性】： negative
posi_score： 5.8 ／ nega_score： 9.4 ／followers： 4169


…………　類似ツイート4位：類似度 0.6　…………

ご飯が出てくるのをいい子にして待ってますみたいな顔してるけど、そもそもそこはあなたの席ではありません https://t.co/o67

In [ ]:
#.pyファイル作成用コマンド
import subprocess
subprocess.run(['jupyter', 'nbconvert', '--to', 'python', 'predict.ipynb'])